# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686604


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.40s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.61s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:27,  1.15s/it]

Rendering models:  27%|██▋       | 8/30 [00:07<00:18,  1.20it/s]

...

Rendering models:  63%|██████▎   | 19/30 [00:07<00:02,  3.94it/s]

Rendering models:  70%|███████   | 21/30 [00:08<00:01,  4.99it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  6.47it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.25it/s]

not-logged-in-840ddeb2efc43eb3b360    0.000686
MissDonkey                            0.000285
zac8151                               0.000446
not-logged-in-f6fac0763296d5759c01    0.015171
ElisabethB                            0.084817
janinecr                              0.003397
SavinSpaceEngineer                    0.001842
mkehrli                               0.064126
austingasbarra                        0.000660
AdamMorse                             0.000234
peakscience7                          0.001641
pangeli5                              0.000804
Lavadude                              0.125109
crush202020                           0.000322
ShirleyL                              0.000773
Bambura                               0.013174
not-logged-in-e9900be6759309fd7692    0.001127
mcooke27                              0.000765
not-logged-in-257beb46445de6db16fb    0.000323
UnbiasedBrigade                       0.001553
h4ranek                               0.002360
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())